# Introduction

This is a training script for a diffusion model called MinImagen. A smaller adaptation of original Imagen architecture introduced by Google.

# Setup

In [ ]:
#install the minimagen package
!pip install minimagen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.3/362.3 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 71.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 

In [ ]:
#utility imports
import os
from datetime import datetime

#pytorch related imports
import torch.utils.data as data_utils
from torch import optim

#minimagen related imports
from minimagen.Imagen import Imagen
from minimagen.Unet import Unet, Base, Super, BaseTest, SuperTest
from minimagen.generate import load_minimagen, load_params
from minimagen.t5 import get_encoded_dim
from minimagen.training import get_minimagen_parser, ConceptualCaptions, get_minimagen_dl_opts, \
    create_directory, get_model_size, save_training_info, get_default_args, MinimagenTrain, \
    load_testing_parameters

ModuleNotFoundError: ignored

In [ ]:
# Get device: Connect to GPU runtime for better performance
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Command line argument parser
parser = get_minimagen_parser()
class args_cls:
  a = 0

#get an instance of the args_cls
args = args_cls()

In [ ]:
#directory creation for training
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
dir_path = f"./training_{timestamp}"
training_dir = create_directory(dir_path)

In [ ]:
#A dictionary of hyperparameters
hyperparameters = dict(
            PARAMETERS=None,
            NUM_WORKERS=0,
            BATCH_SIZE=20,
            MAX_NUM_WORDS=32,
            IMG_SIDE_LEN=128,
            EPOCHS=10,
            T5_NAME='t5_small',
            TRAIN_VALID_FRAC=0.5,
            TRAINING_DIRECTORY = '/content/training_20230731_061334',
            TIMESTEPS=25,
            OPTIM_LR=0.0001,
            ACCUM_ITER=1,
            CHCKPT_NUM=500,
            VALID_NUM=None,
            RESTART_DIRECTORY=None,
            TESTING=False,
            timestamp=None,
        )
# Replace relevant values in arg dict
args.__dict__ = {**args.__dict__, **hyperparameters}

# Data

In [ ]:
# Load subset of Conceptual Captions dataset.
train_dataset, valid_dataset = ConceptualCaptions(args, smalldata=False)
indices = torch.arange(1000)

#create train and validation datasets with given number of samples
train_dataset = data_utils.Subset(train_dataset, indices)
valid_dataset = data_utils.Subset(valid_dataset, indices)

# Create dataloaders
dl_opts = {**get_minimagen_dl_opts(device), 'batch_size': args.BATCH_SIZE, 'num_workers': args.NUM_WORKERS}
train_dataloader = torch.utils.data.DataLoader(train_dataset, **dl_opts)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, **dl_opts)

  0%|          | 0/2 [00:00<?, ?it/s]

# UNet

In [ ]:
# Instantiate Unet with default parameters and transfer to GPU if available
unets_params = [get_default_args(BaseTest), get_default_args(SuperTest)]
unets = [Unet(**unet_params).to(device) for unet_params in unets_params]

In [ ]:
# Specify MinImagen parameters
imagen_params = dict(
    image_sizes=(int(args.IMG_SIDE_LEN / 2), args.IMG_SIDE_LEN),
    timesteps=args.TIMESTEPS,
    cond_drop_prob=0.15,
    text_encoder_name=args.T5_NAME
)

# Create MinImagen from UNets with specified imagen parameters
imagen = Imagen(unets=unets, **imagen_params).to(device)

In [ ]:
# Fill in unspecified arguments with defaults
unets_params = [{**get_default_args(Unet), **i} for i in unets_params]
imagen_params = {**get_default_args(Imagen), **imagen_params}

# Get the size of the Imagen model in megabytes
model_size_MB = get_model_size(imagen)

# Save all training info (config files, model size, etc.)
save_training_info(args, timestamp, unets_params, imagen_params, model_size_MB, training_dir)

# Training

In [ ]:
# Create optimizer - Adam
optimizer = optim.Adam(imagen.parameters(), lr=args.OPTIM_LR)

# Train the MinImagen instance
MinimagenTrain(timestamp, args, unets, imagen, train_dataloader, valid_dataloader, training_dir, optimizer, timeout=30)


-------------------- EPOCH 1 --------------------

----------Training...----------


0it [00:00, ?it/s]


----------Validation...----------



100%|██████████| 5/5 [01:06<00:00, 13.20s/it]
1it [01:14, 74.38s/it]

Unet 0 avg validation loss:  tensor(1.1316, device='cuda:0')
Unet 1 avg validation loss:  tensor(1.0483, device='cuda:0')


...

0it [00:00, ?it/s]


----------Validation...----------



100%|██████████| 5/5 [00:50<00:00, 10.04s/it]
1it [01:37, 97.50s/it]

Unet 0 avg validation loss:  tensor(0.9483, device='cuda:0')
Unet 1 avg validation loss:  tensor(0.9955, device='cuda:0')


5it [02:03, 24.66s/it]


# Inference

In [ ]:
from argparse import ArgumentParser
from minimagen.generate import load_minimagen, sample_and_save


In [ ]:
# Specify the caption(s) to generate images for
captions = ['happy']

In [ ]:
args_cls

__main__.args_cls

In [ ]:
# Use `sample_and_save` to generate and save the iamges
sample_and_save(captions, training_directory='/content/training_20230731_065902')

0it [00:00, ?it/s]
sampling loop time step: 100%|██████████| 25/25 [00:01<00:00, 19.65it/s]
1it [00:01,  1.28s/it]
sampling loop time step: 100%|██████████| 25/25 [00:02<00:00, 11.27it/s]
2it [00:03,  1.76s/it]
